<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Pheme_Veracity_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import fastai

In [2]:
from fastai.text import * 

In [3]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [4]:
# For veracity
train = read_veracity('modifiedTrain_New.csv', True)
test = read_veracity('modifiedTest_New.csv')

In [5]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [6]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

In [7]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [8]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [9]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.571191,4.287068,0.289586,03:45


In [11]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.058862,3.984272,0.326528,04:23


In [12]:
learn.predict("This is a review about", n_words=10)

'This is a review about the police dept . Officers officers have allowed versus'

In [13]:
learn.save_encoder('ft_enc')

In [14]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (76440 items)
x: TextList
xxbos @lolitasaywhat xxunk xxmaj you nailed it . xxmaj they are slowly trying to " xxmaj language xxmaj xxunk " the xxmaj murder of an innocent black man > xxup its xxup murder,xxbos @postbaron @washingtonpost @farhip good article . xxmaj as a writer i thought of this myself .,xxbos xxup @abc xxmaj please xxup rt - xxmaj download a hilarious xxup free xxmaj koran xxup parody for infidels of all ages - https : / / t.co / hxbezhdguj,xxbos @patcondell he 's a great supporter of freedom of expression ...,xxbos @jwademoses @reuterslive @reuters xxmaj there 's two ?
y: CategoryList
False,False,False,False,True
Path: .;

Valid: LabelList (19111 items)
x: TextList
xxbos xxunk @3chicspolitico @quadcitypat if they do nt xxunk negative view of history they should have head xxunk,xxbos @ajenglish 
  xxmaj financial xxmaj times : xxmaj explaining xxmaj bahrain 's land xxunk controversy on xxmaj xxunk - http : / / t.co / 

In [15]:
data_clas.show_batch()

text,target
xxbos .@lisabloom wear your best i ❤ ️ xxmaj surveillance tee \n xxup interntnl xxup wave xxup at xxup surveillance xxup day tomorrow 🙋 \n http : / / t.co / lvnpqfjuw9 \n @till_you @lisabloom xxmaj you did n't get any facts or proof as all this would prove is that he stole cigars . xxmaj has no relation on the death . xxunk \n @androoshaw,True
"xxbos "" @theepharoah : xxup please xxup dont xxup try xxup to xxup buy xxup me xxup out , xxup money xxup or xxup anything . xxup my xxup story xxup is xxup for xxup justice . # xxmaj ferguson http : / / t.co / xxunk """,False
xxbos xxmaj the xxmaj image xxmaj that xxmaj everyone xxmaj is xxmaj sharing is n't xxmaj by xxmaj banksy — xxmaj but xxmaj it 's xxmaj awesome . # charliehebdo http : / / t.co / xxunk http : / / t.co / xxunk,True
"xxbos xxmaj developing : xxmaj hostage situation in xxmaj sydney , xxmaj australia . \n xxmaj updates : http : / / t.co / xxunk \n xxmaj video : http : / / t.co / xxunk http : / / t.co / xxunk",True
xxbos “ @imranhsarker : xxmaj this is the police presence at a peaceful protest - in the xxup usa . # xxmaj ferguson right xxup now ! ! xxup wtf ! ! ! http : / / t.co / xxunk ”,False


In [16]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.540106,0.501679,0.759458,04:06


In [17]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.469731,0.428619,0.799435,04:15


In [18]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.462756,0.405070,0.815813,09:49
1,0.426209,0.353482,0.837057,09:38
2,0.355855,0.316450,0.861284,09:00


Buffered data was truncated after reaching the output size limit.

In [19]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [20]:
from sklearn.metrics import accuracy_score, f1_score

In [21]:
accuracy_score(test['label'], l)

0.5021162842503898

In [22]:
f1_score(test['label'], l, average='macro')

0.4940153557781006

In [23]:
learn.export('veracity.pkl')